In [58]:
import json
from tqdm import tqdm
import random

In [3]:
with open("./AllPrintings.json", "r") as f:
    data = json.load(f)

dict_keys(['meta', 'data'])


In [12]:
print(data.keys())
print(data["data"].keys())
print(data["data"]["10E"].keys())
print(data["data"]["10E"]["cards"][0].keys())

dict_keys(['meta', 'data'])
dict_keys(['10E', '2ED', '2X2', '2XM', '30A', '3ED', '40K', '4BB', '4ED', '5DN', '5ED', '6ED', '7ED', '8ED', '9ED', 'A25', 'AAFR', 'ABRO', 'ACLB', 'ACMM', 'ACR', 'ADMU', 'AER', 'AFC', 'AFR', 'AJMP', 'AKH', 'AKHM', 'AKR', 'ALA', 'ALCI', 'ALL', 'ALTR', 'AMH1', 'AMH2', 'AMID', 'AMKM', 'AMOM', 'ANA', 'ANB', 'ANEO', 'AONE', 'APC', 'ARB', 'ARC', 'ARN', 'ASNC', 'ASTX', 'ATH', 'ATQ', 'AVOW', 'AVR', 'AWOE', 'AZNR', 'BBD', 'BCHR', 'BFZ', 'BIG', 'BLB', 'BNG', 'BOK', 'BOT', 'BRB', 'BRC', 'BRO', 'BRR', 'BTD', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'CC1', 'CC2', 'CED', 'CEI', 'CHK', 'CHR', 'CLB', 'CLU', 'CM1', 'CM2', 'CMA', 'CMB1', 'CMB2', 'CMD', 'CMM', 'CMR', 'CN2', 'CNS', 'CON', 'CP1', 'CP2', 'CP3', 'CSP', 'CST', 'DBL', 'DD1', 'DD2', 'DD3', 'DDC', 'DDD', 'DDE', 'DDF', 'DDG', 'DDH', 'DDI', 'DDJ', 'DDK', 'DDL', 'DDM', 'DDN', 'DDO', 'DDP', 'DDQ', 'DDR', 'DDS', 'DDT', 'DDU', 'DGM', 'DIS', 'DKA', 'DKM', 'DMC', 'DMR', 'DMU', 'DOM', 'DPA', 'DRB', 'DRK',

In [54]:
cards = []
for set_name, set_data in tqdm(data["data"].items()):
    for card in set_data["cards"]:
        cards.append(card)

for k, v in cards[0].items():
    print(f"{k:20} {v}")

100%|██████████| 746/746 [00:00<00:00, 46152.44it/s]

artist               Pete Venters
artistIds            ['d54c4a1a-c0c5-4834-84db-125d341f3ad8']
availability         ['mtgo', 'paper']
boosterTypes         ['default']
borderColor          black
colorIdentity        ['W']
colors               ['W']
convertedManaCost    7.0
edhrecRank           21097
edhrecSaltiness      0.14
finishes             ['nonfoil']
foreignData          [{'flavorText': '„Es ist der Wille aller, und meine Hand, die ihn ausführt."', 'language': 'German', 'multiverseId': 148411, 'name': 'Ausgewählter der Ahnfrau', 'text': 'Erstschlag (Diese Kreatur fügt Kampfschaden vor Kreaturen ohne Erstschlag zu.)\nWenn der Ausgewählte der Ahnfrau ins Spiel kommt, erhältst du 1 Lebenspunkt für jede Karte in deinem Friedhof dazu.', 'type': 'Kreatur — Mensch, Kleriker'}, {'flavorText': '"La voluntad de todos, realizada por mi mano."', 'language': 'Spanish', 'multiverseId': 150317, 'name': 'Elegido de la Antepasada', 'text': 'Daña primero. (Esta criatura hace daño de combate antes

In [52]:
class MtgCard:
    artist: str
    colors: list[str]
    convertedManaCost: float
    edhrecRank: int
    edhrecSalt: float
    legalities: dict[str, bool]
    manaCost: str
    name: str
    ability_text: str
    type_line: str
    power: int
    toughness: int
    loyalty: int
    rarity: str

    def __init__(self) -> None:
        pass

    @staticmethod
    def from_json(card: dict) -> "MtgCard":
        c = MtgCard()
        c.artist = card.get("artist", "")
        c.colors = card["colors"]
        c.convertedManaCost = card["convertedManaCost"]
        c.edhrecRank = card.get("edhrecRank", float("inf"))
        c.edhrecSalt = card.get("edhrecSaltiness", 0.0)
        
        c.legalities = {}
        for k, v in card["legalities"].items():
            c.legalities[k] = (v != "legal")

        c.manaCost = card.get("manaCost", card.get("manaValue", 0))
        c.name = card["name"]
        c.ability_text = card.get("text", "")
        c.type_line = card["type"]

        c.power = card.get("power", None)
        try:
            c.power = int(c.power)
        except:
            c.power = c.power
        c.toughness = card.get("toughness", None)
        try:
            c.toughness = int(c.toughness)
        except:
            c.toughness = c.toughness
        c.loyalty = card.get("loyalty", None)
        try:
            c.loyalty = int(c.loyalty)
        except:
            c.loyalty = c.loyalty
            
        c.rarity = card["rarity"]
        return c
    
    def __str__(self) -> str:
        return f"{self.name} ({self.manaCost})"
    
    def __repr__(self) -> str:
        return str(self)
    
    def __hash__(self) -> int:
        return hash(f"{self.name} {self.manaCost}")
    
    def __eq__(self, o: object) -> bool:
        if not isinstance(o, MtgCard):
            return False
        return self.name == o.name and self.manaCost == o.manaCost

In [68]:
def manaCost_to_cmc(manaCost: str) -> int:
    cmc = 0
    for char in manaCost:
        if char.isdigit():
            cmc += int(char)
        elif char in "}{X":
            pass
        else:
            cmc += 1
    return cmc

In [55]:
card_objs = []
for card in tqdm(cards):
    try:
        card_objs.append(MtgCard.from_json(card))
    except Exception as e:
        for k, v in card.items():
            print(f"{k:20} {v}")
        print(e)
        break

cards = list(set(card_objs))

100%|██████████| 91767/91767 [00:00<00:00, 173622.27it/s]


In [56]:
with open("cards.json", "w") as f:
    json.dump([c.__dict__ for c in cards], f, indent=4)

In [73]:
ambigous_cards: list[tuple[tuple, int]] = [] # (type_line + ability_text + power + toughness + loyalty, manaCost)
for card in tqdm(cards):
    name = card.name
    card_info = [
        card.type_line,
        card.ability_text,
        card.power,
        card.toughness,
        card.loyalty
    ]
    # replace all instances of the card name with "CARDNAME"
    for i in range(len(card_info)):
        if type(card_info[i]) == str:
            card_info[i] = card_info[i].replace(name, "CARDNAME")

    try:
        cmc = manaCost_to_cmc(card.manaCost)
    except:
        cmc = int(card.manaCost)
    ambigous_cards.append((tuple(card_info), cmc))

100%|██████████| 29646/29646 [00:00<00:00, 197063.54it/s]


In [77]:
print(random.choice(ambigous_cards))

(('Legendary Creature — Kirin Spirit', "Flying\nWhenever you cast a Spirit or Arcane spell, you may have target player mill X cards, where X is that spell's mana value.", 4, 4, None), 5)


In [78]:
ambigous_cards_json = []
for card, cmc in ambigous_cards:
    ambigous_cards_json.append({
        "card": card,
        "cmc": cmc
    })

with open("ambigous_cards.json", "w") as f:
    json.dump(ambigous_cards_json, f, indent=4)